In [ ]:
import random
import math
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
class Ant:
    def __init__(self, num_cities):
        self.num_cities = num_cities
        self.tour = []
        self.visited = [False] * num_cities
        self.total_distance = 0.0

    def calculate_tour_distance(self, distance_matrix):
        distance = distance_matrix[self.tour[-1]][self.tour[0]]

        for i in range(self.num_cities - 1):
            distance += distance_matrix[self.tour[i]][self.tour[i + 1]]

        self.total_distance = distance

In [ ]:
class ACO:
    def __init__(self, num_ants, num_iterations, alpha, beta, rho, q):
        self.num_ants = num_ants
        self.num_iterations = num_iterations
        self.alpha = alpha
        self.beta = beta
        self.rho = rho
        self.q = q

    def run(self, distance_matrix):
        num_cities = len(distance_matrix)
        pheromone_matrix = [[1.0] * num_cities for _ in range(num_cities)]
        best_distance = float('inf')
        best_tour = []

        for iteration in range(1, self.num_iterations + 1):
            ants = [Ant(num_cities) for _ in range(self.num_ants)]

            for ant in ants:
                ant.tour.append(random.randint(0, num_cities - 1))
                ant.visited[ant.tour[0]] = True

                for _ in range(num_cities - 1):
                    self.choose_next_city(ant, pheromone_matrix, distance_matrix)

                ant.total_distance += distance_matrix[ant.tour[-1]][ant.tour[0]]

                if ant.total_distance < best_distance:
                    best_distance = ant.total_distance
                    best_tour = ant.tour

            delta_pheromone = self.calculate_delta_pheromone(ants, distance_matrix)

            for i in range(num_cities):
                for j in range(num_cities):
                    pheromone_matrix[i][j] = pheromone_matrix[i][j] * self.rho + delta_pheromone[i][j]

            if iteration >= 10 and iteration % 10 == 0:
                print(f"Iteration {iteration}:")
                self.print_pheromone_map(pheromone_matrix, num_cities)
                print(f"Optimal Path: {best_tour}")
                print()

        return best_tour, best_distance


    def choose_next_city(self, ant, pheromone_matrix, heuristic_matrix):
        current_city = ant.tour[-1]
        unvisited_cities = [i for i in range(ant.num_cities) if not ant.visited[i]]

        probabilities = []
        total_prob = 0.0

        for city in unvisited_cities:
            pheromone = pheromone_matrix[current_city][city]
            heuristic = heuristic_matrix[current_city][city]
            prob = math.pow(pheromone, self.alpha) * math.pow(heuristic, self.beta)
            probabilities.append((city, prob))
            total_prob += prob

        probabilities = [(city, prob / total_prob) for city, prob in probabilities]
        probabilities.sort(key=lambda x: x[1], reverse=True)

        selected_city = self.roulette_wheel_selection(probabilities)
        ant.tour.append(selected_city)
        ant.visited[selected_city] = True
        ant.total_distance += heuristic_matrix[current_city][selected_city]

    def roulette_wheel_selection(self, probabilities):
        r = random.random()
        partial_sum = 0.0

        for city, prob in probabilities:
            partial_sum += prob
            if partial_sum >= r:
                return city

    def calculate_delta_pheromone(self, ants, distance_matrix):
        num_cities = len(distance_matrix)
        delta_pheromone = [[0.0] * num_cities for _ in range(num_cities)]

        for ant in ants:
            for i in range(num_cities - 1):
                city1, city2 = ant.tour[i], ant.tour[i + 1]
                delta_pheromone[city1][city2] += self.q / ant.total_distance

        return delta_pheromone

    def calculate_heuristic_matrix(self, distance_matrix):
        num_cities = len(distance_matrix)
        heuristic_matrix = [[0.0] * num_cities for _ in range(num_cities)]

        for i in range(num_cities):
            for j in range(num_cities):
                if i != j:
                    heuristic_matrix[i][j] = 1.0 / distance_matrix[i][j]

        return heuristic_matrix
    def print_pheromone_map(self, pheromone_matrix, num_cities):
        print("Pheromone Map:")
        print("            ", end="")
        for j in range(num_cities):
            print(f"{j:8}", end="")
        print()

        for i in range(num_cities):
            print(f"{i:4}        ", end="")
            for j in range(num_cities):
                print(f"{pheromone_matrix[i][j]:.4f}  ", end="")
            print()


In [ ]:
# Generate distance matrix
def generate_distance_matrix(num_cities):
    distance_matrix = [[0] * num_cities for _ in range(num_cities)]

    for i in range(num_cities):
        for j in range(i + 1, num_cities):
            distance = random.randint(3, 50)
            distance_matrix[i][j] = distance
            distance_matrix[j][i] = distance

    return distance_matrix

In [ ]:
# Example usage
num_cities = 10
num_ants_list = [1, 5, 10, 20]
num_iterations = 50
alpha = 1.0
beta = 2.0
rho = 0.5
q = 100.0

for num_ants in num_ants_list:
    distance_matrix = generate_distance_matrix(num_cities)
    aco = ACO(num_ants, num_iterations, alpha, beta, rho, q)
    best_tour, best_distance = aco.run(distance_matrix)

    print(f"Number of ants: {num_ants}")
    print(f"Best tour: {best_tour}")
    print(f"Best distance: {best_distance}")
    print()

Iteration 10:
Pheromone Map:
                   0       1       2       3       4       5       6       7       8       9
   0        0.0010  0.0186  0.0366  0.0010  0.0010  0.0010  0.0023  0.0010  0.0010  0.4219  
   1        0.0023  0.0010  0.0010  0.0010  0.0010  0.0010  0.0055  0.0015  0.5476  0.0010  
   2        0.0010  0.5239  0.0010  0.0010  0.0010  0.0144  0.0186  0.0010  0.0010  0.0010  
   3        0.0055  0.0010  0.0010  0.0010  0.0010  0.2875  0.2379  0.0283  0.0010  0.0010  
   4        0.0010  0.0029  0.2921  0.0010  0.0010  0.0010  0.0010  0.0032  0.0010  0.1267  
   5        0.0010  0.0093  0.0854  0.0010  0.0010  0.0010  0.0010  0.1354  0.0010  0.0096  
   6        0.4971  0.0010  0.0010  0.0055  0.0366  0.0010  0.0010  0.0010  0.0023  0.0010  
   7        0.0010  0.0010  0.0010  0.0010  0.5206  0.0339  0.0010  0.0010  0.0037  0.0010  
   8        0.0010  0.0055  0.0010  0.5300  0.0010  0.0221  0.0010  0.0010  0.0010  0.0010  
   9        0.0208  0.0010  0.0023  0.001

In [ ]:
# Example usage
num_cities = 20
num_ants_list = [1, 5, 10, 20]
num_iterations = 50
alpha = 1.0
beta = 2.0
rho = 0.5
q = 100.0

for num_ants in num_ants_list:
    distance_matrix = generate_distance_matrix(num_cities)
    aco = ACO(num_ants, num_iterations, alpha, beta, rho, q)
    best_tour, best_distance = aco.run(distance_matrix)

    print(f"Number of ants: {num_ants}")
    print(f"Best tour: {best_tour}")
    print(f"Best distance: {best_distance}")
    print()

Iteration 10:
Pheromone Map:
                   0       1       2       3       4       5       6       7       8       9      10      11      12      13      14      15      16      17      18      19
   0        0.0010  0.0010  0.0010  0.0010  0.0671  0.0010  0.0010  0.0010  0.0010  0.0010  0.0021  0.1659  0.0010  0.0265  0.0053  0.0010  0.0010  0.0016  0.0010  0.0010  
   1        0.0010  0.0010  0.0027  0.0010  0.0223  0.0010  0.0010  0.0010  0.0010  0.0033  0.2380  0.0010  0.0010  0.0010  0.0010  0.0010  0.0010  0.0012  0.0010  0.0010  
   2        0.0010  0.0010  0.0010  0.0674  0.0010  0.0010  0.0010  0.0147  0.0010  0.0010  0.0010  0.0016  0.0010  0.0010  0.1806  0.0010  0.0010  0.0033  0.0010  0.0010  
   3        0.0010  0.0010  0.0053  0.0010  0.0021  0.0010  0.0010  0.0033  0.0010  0.0010  0.0180  0.0012  0.0010  0.2298  0.0098  0.0010  0.0010  0.0010  0.0010  0.0010  
   4        0.0010  0.0016  0.0010  0.1719  0.0010  0.0033  0.0010  0.0010  0.0053  0.0010  0.0010  0.0010